# ACP de visages

Nous allons faire une ACP sur des images. On considérera pour cela que nos images sont des vecteurs où chaque pixel est un entier entre 0 et 255 (en niveaux de gris).

## Chargement des données

Le chargement des données MNIST s'effectue facilement grâce aux commandes suivantes. Le site associé est http://vis-www.cs.umass.edu/lfw/

**Attention** : la 1ère fois, la ligne suivante va télécharger 200 méga de données, et créer un dossier `lfw_home`.

On va télécharger les images des personnalités ayant plus de 20 images ([doc de la fonction](https://scikit-learn.org/stable/modules/generated/sklearn.datasets.fetch_lfw_people.html))

In [ ]:
from sklearn.datasets import fetch_lfw_people

lfw_people = fetch_lfw_people(data_home=".", min_faces_per_person=20, resize=0.4)

Quelques variables associées aux images :

In [ ]:
images = lfw_people.data #liste d'images
nombre_images, hauteur_image, largeur_image = lfw_people.images.shape

print("nombre images : ", nombre_images)
print("dimension des images : ", hauteur_image, largeur_image)

Les personnalités : 

In [ ]:
personnalités = lfw_people.target_names

for nom in personnalités:
    print(nom)

In [ ]:
personnalités_index = [personnalités[lfw_people.target[index]] for index in range(nombre_images)]

for index in range(nombre_images):
    print("image", index, "nom :", personnalités_index[index])

## Regardons les images

Chaque image est une liste de coordonnées. 
Nos donnees sont ainsi des vecteurs de dimension hauteur * largeur. Chaque coordonnée est un niveau de gris.

Regardans l'image d'index 2698

In [ ]:
print(images[2698])

Pour représenter graphiquement cette ligne, il faut reconstruire l'image en la séparant en ligne :

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt

sns.reset_orig() # pour une fois on ne veut pas des paramètres de seaborn
current_palette = sns.color_palette()

In [ ]:
fig, ax = plt.subplots(figsize=(3, 3)) 

plt.imshow(images[2698].reshape((hauteur_image, largeur_image)), 
           cmap=plt.cm.gray)

plt.title(personnalités_index[2698])

plt.show()

**QUESTION** : Trouvez l'index de toutes les photos de winona ryder.

In [ ]:
for index in range(nombre_images):
    if personnalités_index[index] == "Winona Ryder":
        print("image", index)

## l'ACP

Centrer et reduire les donnees a normalisé les niveaux de gris pour toutes les images. On est pas obligé de le faire, mais si on ne le fait pas, le calcul des $cos^2$ va être faux, il ne faudra pas en tenir compte.

On utilisera maintenant les données centrées et réduites. Elles sont un peut différentes des images de départ. Regardez commant Winona a été changée.

In [ ]:
images

In [ ]:
from sklearn.preprocessing import StandardScaler
import pandas

In [ ]:
scaler = StandardScaler()

images_scaled = scaler.fit_transform(images)

In [ ]:
fig, ax = plt.subplots(figsize=(3, 3)) 

plt.imshow(images_scaled[2698].reshape((hauteur_image, largeur_image)), 
           cmap=plt.cm.gray)

plt.title(personnalités_index[2698])

plt.show()

Effectuons une analyse en composantes principale de nos images.

Comme le nombre de dimensions est très importante, nous ne calculerons que les 50 premiers axes. On choisi le nombre d'axe à conserver en utilisant le paramètre `n_components` de la fonction `PCA` de sklearn (voir https://scikit-learn.org/stable/modules/generated/sklearn.decomposition.PCA.html)

In [ ]:
import pandas
from sklearn.decomposition import PCA 
import numpy as np

In [ ]:
X = pandas.DataFrame(images_scaled)

pca = PCA(n_components=50)
pca.fit(X)

U = np.transpose(pca.components_) # vecteurs propres
I = pandas.DataFrame(np.transpose(pca.explained_variance_ratio_), columns=["pourcentage"])  # information véhiculée

C = pandas.DataFrame(X @ U, index=X.index) # nouvelles coordonnées

corrélations = pandas.DataFrame([[C[facteur].corr(X[column]) for facteur in C] for column in X], index=X.columns)
cos2 = (C**2).div((X**2).sum(axis=1), axis='index')

**QUESTION** : Affichez l'inertie du nuage

In [ ]:
I

In [ ]:
fig, ax = plt.subplots(figsize=(7, 7)) 

I.plot(ax=ax)

plt.title("inertie expliquée")
plt.xlabel("axes principaux")
plt.ylabel("% d'inertie expliquée")

plt.show() 


* regarder l'inertie cumulée. Combien d'inertie est conservée sur les 50 permiers axes ?
* combien d'axes avons nous en tout ?
* la qualité de la représentation (l'angle) pour les 50 axes gardés 
* pour les 12 premiers axes

In [ ]:
I.cumsum()

In [ ]:
fig, ax = plt.subplots(figsize=(7, 7)) 

I.cumsum().plot(ax=ax)

plt.title("inertie expliquée cumulée")
plt.xlabel("axes principaux")
plt.ylabel("% d'inertie expliquée")

plt.show() 

In [ ]:
I.sum()

On a conservé 85% de l'inertie pour 50 axes. Il y a `hauteur_image * largeur_image` axes en tout.

Qualité de projection ($cos^2$) :

In [ ]:
cos2 # cos^2

Qualité de projection (angles) :

In [ ]:
#angles 
from math import sqrt, acos, pi

cos2.sum(axis=1).apply(lambda x: acos(sqrt(x)) * 180 / pi).sort_values()

Les plus proches ont un angle de 9°, les plus éloignées de 45°

In [ ]:
cos2.loc[:, :11].sum(axis=1).apply(lambda x: acos(sqrt(x)) * 180 / pi).sort_values()

Les plus proches ont un angle de °12, les plus éloignées de 65°

## On peut regarder les nouveaux axes (eigenfaces)

On a gardé les 50 premiers vecteur propres. Chacun étant un vecteur de dimension  hauteur * largeur :

In [ ]:
hauteur_image * largeur_image

Le premier vecteur est une colonne de U. Pour l'obtenir :

In [ ]:
U.transpose()[0]

**QUESTION** : Représentez l'image associée à cet axe

In [ ]:
fig, ax = plt.subplots(figsize=(7, 7)) 

plt.imshow(U.transpose()[0].reshape((hauteur_image, largeur_image)), 
           cmap=plt.cm.gray)

plt.show()

**QUESTION** : Représentez les images associées aux 12 premiers axes

In [ ]:
fig, ax = plt.subplots(figsize=(7, 7), nrows=3, ncols=4) 

for i in range(3):
    for j in range(4):
        ax[i][j].set_xticks([])
        ax[i][j].set_yticks([])
        ax[i][j].set_title("axe " + str(i * 4 + j))
        ax[i][j].imshow(U.transpose()[4 * i + j].reshape((hauteur_image, largeur_image)), 
                                  cmap=plt.cm.gray)

plt.show()

## Les images approximées

On peu reconstruire les images en n'utilisant que les axes considérés. Nous en avons pris 50 sur les hauteur * largeur possibles.

In [ ]:
images_scaled

Pour ne conserver que les 50 premiers axes, on pourrait faire le calcul en reprojetant nos nouvelles coordonnées sur l'ancien repère, mais la sklearn nous permet de le faire tout seul.

**QUESTION** : Quelle est la qualité de représentation ($cos^2$ et angle) de l'image 2698 ?

In [ ]:
#cos^2

cos2.sum(axis=1)[2698]

In [ ]:
#angle

acos(sqrt(cos2.sum(axis=1)[2698]))

Winona dans le nouveau repère des 50 premiers axes factoriels :

In [ ]:
C.iloc[2698]

Dans le repère originel :

In [ ]:
image_reconstruite = pca.inverse_transform(C.iloc[2698])

image_reconstruite

**QUESTION** : représentez l'image reconstruite. Mettez côte à côte l'image originelle, l'image centrée réduite et l'image sur les 50 premiers axes factoriels.

In [ ]:
fig, ax = plt.subplots(figsize=(7, 7), nrows=1, ncols=3) 

ax[0].set_xticks([])
ax[0].set_yticks([])
ax[0].set_title("originale")
ax[0].imshow(images[2698].reshape((hauteur_image, largeur_image)), 
             cmap=plt.cm.gray)

ax[1].set_xticks([])
ax[1].set_yticks([])
ax[1].set_title("centrée réduite")
ax[1].imshow(images_scaled[2698].reshape((hauteur_image, largeur_image)), 
             cmap=plt.cm.gray)

ax[2].set_xticks([])
ax[2].set_yticks([])
ax[2].set_title("projetée")
ax[2].imshow(image_reconstruite.reshape((hauteur_image, largeur_image)), 
             cmap=plt.cm.gray)


plt.show()

## Nombre d'axes utilisés

On ne va pas centrer/reduire les données et voir l'évolution d'une image lorsque l'on ajoute des axes.

Pour cela, commançons par faire une ACP sur 200 composantes (attention, ça va prendre du temps) :

In [ ]:
X_img = pandas.DataFrame(images)

pca = PCA(n_components=200)
pca.fit(X_img)

U_img = np.transpose(pca.components_) # vecteurs propres
I_img = pandas.DataFrame(np.transpose(pca.explained_variance_ratio_), columns=["pourcentage"])  # information véhiculée

C_img = pandas.DataFrame(X_img @ U_img, index=X.index) # nouvelles coordonnées

corrélations_img = pandas.DataFrame([[C[facteur].corr(X[column]) for facteur in C] for column in X], index=X.columns)
cos2_img = (C**2).div((X**2).sum(axis=1), axis='index')

Les coordonnées d'une image selon les nouveaux axes dont données par `C_img` et les coordonnées des nouveaux axes dans l'ancien repère est donnée par `U_img`.

Les coodonnées de l'image 2698 dans le repère original est alors, pour les 200 axes :

In [ ]:
np.matmul(U_img, C_img.loc[2698])

In [ ]:
img = np.matmul(U_img, C_img.loc[2698])

fig, ax = plt.subplots(figsize=(7, 7)) 

ax.imshow(img.reshape((hauteur_image, largeur_image)), 
             cmap=plt.cm.gray)
plt.show()

Et pour les 50 premiers axes : 

In [ ]:
img = np.matmul(U_img[:, :50], C_img.loc[2698][:50])

fig, ax = plt.subplots(figsize=(7, 7)) 

ax.imshow(img.reshape((hauteur_image, largeur_image)), 
          cmap=plt.cm.gray)
plt.show()

**Question** Prenez l'image de la base de données d'indice 2698 et affichez sa reconstruction par une acp utilisant 
5, 10, 50, 100 et 200 composantes.

In [ ]:
fig, ax = plt.subplots(figsize=(15, 5), nrows=1, ncols=5) 

for i, nb in zip(range(5), [5, 10, 50, 100, 200]):
    ax[i].set_xticks([])
    ax[i].set_yticks([])
    ax[i].set_title(str(nb))
    img = np.matmul(U_img[:, :nb], C_img.loc[2698][:nb])
    ax[i].imshow(img.reshape((hauteur_image, largeur_image)), 
                 cmap=plt.cm.gray)

plt.show()

## Quels axes sont importants pour une image donnée

**QUESTION** :  Regardez la qualité de la projection de cette image avec les nouveaux axes et trouvez les 12 axes les plus important pour cette image.

In [ ]:
cos2_img.iloc[2698].sort_values(ascending=False).head(12)

**QUESTION** : Dessinez les eigenfaces correspondantes à ces 12 vecteurs.

on utilise le fait que l'on peut passer n'importe quelle ensemble de colonnes comme sélecteur

In [ ]:
cols = cos2_img.iloc[2698].sort_values(ascending=False).head(12).index

In [ ]:
img = np.matmul(U_img[:, cols], C_img.loc[2698][cols])

fig, ax = plt.subplots(figsize=(7, 7)) 

ax.imshow(img.reshape((hauteur_image, largeur_image)), 
          cmap=plt.cm.gray)
plt.show()

le $cos^2$ :

In [ ]:
sum(cos2_img.iloc[2698][cols])

En prenant uniquement les 12 premiers axes, c'est un peu moins bien :

In [ ]:
img = np.matmul(U_img[:, :12], C_img.loc[2698][:12])

fig, ax = plt.subplots(figsize=(7, 7)) 

ax.imshow(img.reshape((hauteur_image, largeur_image)), 
          cmap=plt.cm.gray)
plt.show()

le $cos^2$ :

In [ ]:
sum(cos2_img.iloc[2698][:12])